### Import libraries and file

In [7]:
import pandas as pd
import csv
import numpy as np
import warnings
from scipy.spatial.distance import squareform, pdist

In [3]:
filename = 'apps_permission.csv'

### Function do find the max size of columns

In [4]:
#preprocessing
def get_max_len():
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        num = []
        for i, row in enumerate(reader):
            num.append(len(row))
        m = max(num)
        #print m
        return m
    
print(get_max_len())

126


### Read and show dataset

In [5]:
df = pd.read_csv(filename, header=None, na_values=0, names=range(get_max_len()), low_memory=False)

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,125
0,idteam.app.crazypianofree,android.permission.ACCESS_NETWORK_STATE,android.permission.INTERNET,android.permission.MODIFY_AUDIO_SETTINGS,android.permission.READ_EXTERNAL_STORAGE,android.permission.READ_PHONE_STATE,android.permission.WRITE_EXTERNAL_STORAGE,android.permission.WRITE_INTERNAL_STORAGE,com.android.launcher.permission.INSTALL_SHORTCUT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,com.extremeenjoy.deutschnews,android.permission.ACCESS_NETWORK_STATE,android.permission.INTERNET,android.permission.RECEIVE_BOOT_COMPLETED,android.permission.VIBRATE,android.permission.WAKE_LOCK,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,com.extremeenjoy.news.kannadacinema,android.permission.ACCESS_NETWORK_STATE,android.permission.INTERNET,android.permission.RECEIVE_BOOT_COMPLETED,android.permission.VIBRATE,android.permission.WAKE_LOCK,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,com.vivek.batteryfloater,android.permission.READ_EXTERNAL_STORAGE,android.permission.READ_PHONE_STATE,android.permission.SYSTEM_ALERT_WINDOW,android.permission.WRITE_EXTERNAL_STORAGE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,coolcherrytrees.games.reactor4,android.permission.INTERNET,android.permission.WAKE_LOCK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Filter Dataset

In [7]:
permissions = set()
permissions_list = set()
apps_list = set()


for index, row in df.iterrows():
    app = row[0]
    
    for perm in row[1:]:
        if str(perm) == 'nan':
            break
        else:
            perm_list = perm.split('.')
            if 'permission' in perm_list:
                perm_t = perm_list[-1].lower()
                
                if 'token' not in perm_t and len(perm_t) > 0:
                    permissions.add(perm_t)
                    permissions_list.add((app, perm_t))
                    apps_list.add(app)

### Create new dataframe from filter permissions and apps

In [8]:
df_apps = pd.DataFrame(list(permissions_list))
df_apps.head()

,0,1
0,com.app_deca1.layout,access_fine_location
1,com.zeitmobile.eighttoeight,access_network_state
2,com.cyou.security,mount_unmount_filesystems
3,com.doosan.ebsaudio.ear1108,access_network_state
4,com.rcplatform.collageart,change_configuration


### Create a matrix from dataframe

In [9]:
matrix = pd.crosstab(index=df_apps[0], columns = df_apps[1])
matrix.to_csv('permissions_matrix.csv', index=False)

### Calculate similarity matrix (>= 16gb to run)

In [10]:
similarity_matrix = np.float32(squareform(pdist(matrix, 'cosine')))

### Save similarity matrix in a jupyter-notebook storemagic

In [11]:
#%store similarity_matrix

### Write similarity matrix in file (.npy)

In [12]:
np.save('/mnt/files/sim_matrix.npy', similarity_matrix)

### Write similarity matrix in file (.csv)

In [13]:
np.savetxt("/mnt/files/sim_matrix.csv", similarity_matrix, delimiter=",", fmt='%1.8f')

### Load npy file

In [8]:
similarity_matrix = np.load('/mnt/files/sim_matrix.npy')

In [9]:
from kmedoids import kmedoids

In [10]:
medoid = {}

In [11]:
cluster = {}

In [12]:
for i in range(2,19):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        sm, c = kmedoids(similarity_matrix, k=i)
        medoid[i] = sm
        cluster[i] = c

In [13]:
np.save('/mnt/files/medoids_2_18.npy', medoid)

In [14]:
np.save('/mnt/files/cluster_kmedoids_2_18.npy', cluster)

In [15]:
with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        sm, c = kmedoids(similarity_matrix, k=310)

In [16]:
np.save('/mnt/files/medoids_310.npy', sm)

In [17]:
np.save('/mnt/files/cluster_kmedoids_310.npy', c)

In [15]:
np.save('/mnt/files/cluster_kmedoids_310.npy', c)

In [5]:
from sklearn.cluster import AgglomerativeClustering

In [6]:
hc = AgglomerativeClustering(n_clusters = 10, affinity = 'precomputed', linkage = 'complete')

In [7]:
y_hc = hc.fit(similarity_matrix)

MemoryError: 

In [20]:
del y_hc

NameError: name 'y_hc' is not defined

In [ ]:

    file_name = '/mnt/files/AgglomerativeClustering'+str(i)+'.out'
    np.savetxt(file_name, y_hc.n_clusters, fmt='%d')

In [25]:
pwd

'/home/ec2-user/googleplay-api'

In [ ]:
for i in range(5,310,5):
    hc = AgglomerativeClustering(n_clusters = i, affinity = 'precomputed', linkage = 'complete')
    y_hc = hc.fit(X)
    file_name = '/mnt/files/AgglomerativeClustering'+str(i)+'.out'
    np.savetxt(file_name, y_hc.n_clusters, fmt='%d')